## Electrical routing in IPKISS

Similar to [IPKISS waveguides](waveguide.ipynb), electrical routing in IPKISS can also be created with a **trace template**, and a defined trace (route).
Similar as the previous example, where we use `RouteManhattan` function to create Manhattan-like (orthogonal) waveguide routing between two define ports, here, we will use a metal-based template instead of a waveguide-based template.

In [ ]:
%matplotlib inline
from technologies import silicon_photonics
from ipkiss3 import all as i3
import pylab as plt
import numpy as np
import sys

plt.rcParams['figure.figsize'] = (8, 8)
sys.path.insert(0, '.')

In [ ]:
from ipkiss3.pcell.routing import RouteManhattan
from ipkiss3.pcell.wiring import ElectricalPort, ElectricalWire, ElectricalWireTemplate

input_port = ElectricalPort(name="in", position=(5.0, 5.0))
output_port = ElectricalPort(name="out", position=(20.0, 20.0))

# create the route object
route = RouteManhattan(input_port=input_port,
                       output_port=output_port)

tpl = ElectricalWireTemplate()
wire = ElectricalWire(trace_template=tpl)
layout = wire.Layout(shape=route)
layout.visualize()

Next, let's see an example of using metal routing in a circuit design. In this case, we create a very simple design with two bondpads, and we connect them by a metal wire. We use the standard picazzo PCell [PlaceComponents](http://docs.lucedaphotonics.com/3.1/picazzo/routing/place_route/ref/picazzo3.routing.place_route.cell.PlaceComponents.html) to place several components.

In [ ]:
from technologies import silicon_photonics
from ipkiss3 import all as i3
from picazzo3.routing.place_route import PlaceComponents
from ipkiss3.pcell.wiring import ElectricalPort, ElectricalWire, ElectricalWireTemplate
from support_files.bondpad import BondPad

tpl = ElectricalWireTemplate()

class ElectricalRoutingExample(PlaceComponents):

    bondpad = i3.ChildCellProperty(doc="Bondpad used")
    num_bondpads = i3.PositiveIntProperty(doc="Number of bondspads")
    
    def _default_num_bondpads(self):
        return 5

    def _default_bondpad(self):
        return BondPad()

    def _default_child_cells(self):        
        childs = dict()
        for w in range(self.num_bondpads):
            childs["bp_north_{}".format(w)] = self.bondpad
            childs["bp_west_{}".format(w)] = self.bondpad

        return childs


    class Layout(PlaceComponents.Layout):
        
        bp_spacing = i3.PositiveNumberProperty(default=200.0, doc="Spacing between the bondpads")
        center_bp_north = i3.PositiveNumberProperty(doc="Center for the north bondpads")
        center_bp_west = i3.PositiveNumberProperty(doc="Center for the west bondpads")
        metal_width = i3.PositiveNumberProperty(doc="Metal width", default=5.0)
        

        def _default_center_bp_north(self):
            return 1000

        def _default_center_bp_west(self):
            return 100

        def _default_child_transformations(self):
            trans = dict()
            for w in range(self.num_bondpads):
                trans["bp_north_{}".format(w)] = i3.Rotation(rotation=90.0) + i3.Translation(translation=(self.center_bp_north+(w-self.num_bondpads/2.0+0.5)*self.bp_spacing, 1000-100))
                trans["bp_west_{}".format(w)] = i3.Rotation(rotation=90.0) + i3.Translation(translation=(0 - 100, self.center_bp_west - (w - self.num_bondpads / 2.0 + 0.5) * self.bp_spacing))

            return trans
        
        
        @i3.cache()
        def get_electrical_routes(self):
            routes = []
            trans = self.child_transformations
            for w in range(self.num_bondpads):
                p1 = self.bondpad.ports["m1"].transform_copy(transformation=trans["bp_north_{}".format(w)])
                p2 = self.bondpad.ports["m1"].transform_copy(transformation=trans["bp_west_{}".format(w)]) 
                r = i3.RouteManhattan(input_port=p1, 
                                      output_port=p2,
                                      angle_out=0.0,
                                      angle_in=-90,
                                      rounding_algorithm=None)
                routes.append(r)


            return routes
        
        def _generate_instances(self, insts):
            insts = super(ElectricalRoutingExample.Layout, self)._generate_instances(insts)
            for r in self.get_electrical_routes():
                wire = ElectricalWire(trace_template=tpl)
                layout = wire.Layout(shape=r)
                insts += i3.SRef(layout)
                
            return insts
                
ElectricalRoutingExample().Layout().visualize()


Now, you are introduced to the trace template concepts and you are able to create waveguide and metal wires!